# Test of Trained model.

In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! pip install transformers

In [5]:
from transformers import DistilBertTokenizer, DistilBertConfig, TFDistilBertModel

distil_bert = 'distilbert-base-uncased' # Name of the pretrained models : (Wights)

#DistilBERT 
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model = TFDistilBertModel.from_pretrained(distil_bert)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


## Load the Trained Fully Connected Neural nets.

In [6]:
# Fully connected neural nets which is trained on the extracted features from Bert.
cls_test_model = tf.keras.models.load_model('/content/drive/My Drive/IMDB_review_clf_Model')

In [7]:
# Functions for preprocessing the text data.
import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocess_text(sen):

    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z.!]', ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence    

## Define a function to predict the sentiment of a Review Text.

In [8]:
# Function which will use to predict the sentiment of a review text.

def pred_sentiment(sentence):
  sentence = preprocess_text(sentence)
  if len(sentence) >=512:
    sentence = sentence[:510]
  e = tokenizer.encode(sentence)
  output = model(tf.constant(e)[None,:])
  pred = cls_test_model.predict(output[0][0][0][None,:])
  return pred[0][0]  

### Load the IMDB dataset.

In [9]:
# Loading the IMDB dataset from drive.
import pandas as pd
imdb_data = pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Predict the sentiment of some Review text:

In [13]:
i = 40280
pred = pred_sentiment(imdb_data['review'][i])
print('Actual Sentiment is =',imdb_data['sentiment'][i],'   Probability of being positive =',pred)

Actual Sentiment is = positive    Probability of being positive = 0.99407095


In [14]:
i = 45085
pred = pred_sentiment(imdb_data['review'][i])
print('Actual Sentiment is =',imdb_data['sentiment'][i],'   Probability of being positive =',pred)

Actual Sentiment is = positive    Probability of being positive = 0.77426493


In [15]:
i = 47555
pred = pred_sentiment(imdb_data['review'][i])
print('Actual Sentiment is =',imdb_data['sentiment'][i],'   Probability of being positive =',pred)

Actual Sentiment is = negative    Probability of being positive = 0.099473916


In [16]:
i = 44444
pred = pred_sentiment(imdb_data['review'][i])
print('Actual Sentiment is =',imdb_data['sentiment'][i],'   Probability of being positive =',pred)

Actual Sentiment is = positive    Probability of being positive = 0.87660533
